In [ ]:
import os
from pymystem3 import Mystem
from pyaspeller import YandexSpeller
import pymorphy2
from tqdm import tqdm

In [ ]:
speller = YandexSpeller()
mystem = Mystem(grammar_info=False)
morph = pymorphy2.MorphAnalyzer()

In [ ]:
dict_words = {}
def norm_form(string):
    words = string.split(' ')
    words_norm = []
    for word in words:
        if word in dict_words:
            words_norm.append(dict_words[word])
        else:
            norm = morph.parse(word)[0].normal_form
            words_norm.append(norm)
            dict_words[word] = norm
    return ' '.join(words_norm)

Исправим запросы

In [ ]:
new_lines = []
queries = {}
with open('queries.tsv', encoding='utf-8') as f:
    for line in tqdm(f):
        l = line.strip('\n').split('\t')
        fixed = speller.spelled(l[1])
        fixed = norm_form(fixed)
        queries[l[0]] = fixed
        new_lines.append('\t'.join([l[0], fixed]))


In [ ]:
norm_q_files = open('norm_queries_raw.tsv', 'w')
norm_q_files.writelines(new_lines)
norm_q_files.close()

*norm_queries.txt* - это *norm_queries_raw.txt* после проверки глазками

In [ ]:
#data 
queries = {}
with open('norm_queries.tsv', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
        line = line.strip('\n').split('\t')
        queries[line[0]] = line[1]

Выделим только те документы, которые есть в трейне и тесте

In [ ]:
train_docs=[]
train_queries = []
with open("train.marks.tsv", 'r', encoding='utf-8') as f:
    for line in tqdm(f.readlines()):
        line=line.strip('\n').split('\t')
        train_docs.append(line[1])
        train_queries.append(line[0])

In [ ]:
train_docs = list(set(train_docs))
train_queries = list(set(train_queries))

In [ ]:
test_docs=[]
test_queries = []
with open("sample.csv", 'r', encoding='utf-8') as f:
    f.readline()
    for line in f.readlines():
        line=line.strip('\n').split(',')
        test_queries.append(line[0])
        test_docs.append(line[1])
test_docs = list(set(test_docs))
test_queries = list(set(test_queries))

In [ ]:
all_queries = train_queries + test_queries
all_docs = train_docs + test_docs

Записываем документы в отдельные папки, заодно отдельно сохраняем заголовки, чтобы потом исправить в них опечатки.

In [ ]:
try:
     os.mkdir('data')
except:
     n = None
        
titles = {}
        
with open("docs.tsv", encoding='utf-8') as docs_file:
    for line in tqdm(docs_file):
        l = line.split("\t")
        if l[0] in all_docs:
            titles[l[0]] = l[1]
            #with open('data/'+l[0]+'.txt','w',encoding='utf-8') as file:
            #    file.write('\t'.join(l[1:]))
            #file.close()

Исправляем опечатки в заголовках и приводим слова к нормальной форме

In [ ]:
titles_lines = []

for doc_id in tqdm(titles.keys()):
    fixed = speller.spelled(titles[doc_id])
    fixed = norm_form(fixed)
    titles_lines.append(doc_id + '\t' + fixed + '\n')


In [ ]:
f_titles = open('norm_titles.tsv', 'w')
f_titles.writelines(titles_lines)
f_titles.close()